# Database Reconstruction

[Dinur & Nissim (2003)](http://www.cse.psu.edu/~ads22/privacy598/papers/dn03.pdf), roughly, states that if too many queries on a database are answered too accurately, the database can be reconstructed from the answeries to the queries. Before this fact was properly understood, statistical releases from databases were typically limited in ad-hoc ways. For example, an interactive querying system (in which analysts propose their own queries) may not allow queries that clearly single out individual elements of the data set. Similarly, in a one-shot release (e.g. the U.S. Census Bureau releasing releasing statistical tables) the releasing party may decide not to release statistics on groups with fewer than some fixed number of elements. 

Releasing statistics in a differentially private manner has been posed as a principled way to ensure the privacy of individuals included in a database and prevent database reconstruction. Notably, the U.S. Census is moving to use differentially private algorithms to perform statistical releases for the 2020 Census, for fear that their old methods of privatization (a.k.a. statistical disclosure limitation) are not sufficient to prevent database reconstruction. See [here](https://dl.acm.org/ft_gateway.cfm?id=3295691&ftid=2020934&dwn=1) for more information. 

In this notebook we will focus on the one-shot release model and simulate the process of a data owner releasing aggregate statistics from a sensitive database, both with and without differential privacy. We will then attempt to reconstruct the original database from each set of releases.

### Statistical Release Setup

The underlying sensitive data from which we are releasing is a partly-synthetic version of the PUMS data that are used in many other notebooks. The data are $(1000 \times 6)$ with the columns taking values as follows:

    - educ: {1, 2}
    - race: {1, 2}
    - agebinned: {'[20, 25)', '[25, 30)', ..., '[45, 50)'}
    - sex: {0, 1}
    - married: {0, 1}
    - income: [0, 420,500]

We will assume that `income` is the only sensitive variable in the database, and so the data owner is concerned only with whether or not an attacker can reconstruct the `income` variable.

We will first construct calculate a number of statistics -- `mean`, `median`, `min`, and `max` by subgroup, where a subgroup is defined as a set of database elements that are equivalent with respect to some subset of variables. For example, valid subgroups include the set of all elements with `educ == 2`, and the set of all elements with `educ == 1 && race == 2 && married == 0`. The `income` variable is never included in these subgroup definitions, as it is the variable we are trying to reconstruct. 

Throughout the notebook, we will examine various cases in which the data owner releases different subsets of these statistics.


In [1]:
import reconstruction_module as rec

# load data 
orig_data, data = rec.load_data()
non_income_data = data.drop('income', axis = 1)

# get plausible variable combinations and subset of length 5 plausible combinations 
plausible_variable_combinations = rec.get_plausible_variable_combinations(non_income_data)
plausible_variable_combinations_names = ['__'.join(combination) for combination in plausible_variable_combinations]

five_way_interactions = [combination for combination in plausible_variable_combinations if len(combination) == 5]
five_way_interactions_names = ['__'.join(combination) for combination in five_way_interactions]

# get dictionaries of private and non-private releases (up to 5-way interactions)
count_dict, priv_count_dict, mean_income_dict, priv_mean_income_dict, median_income_dict, priv_median_income_dict, min_income_dict, priv_min_income_dict, max_income_dict, priv_max_income_dict = rec.create_dicts(data, non_income_data, plausible_variable_combinations)

# get string representations of each element associated with each tuple representing the 5-way interactions
elem_dict, priv_elem_dict = rec.create_elem_dicts(count_dict, priv_count_dict, five_way_interactions, five_way_interactions_names)

### Statistical Release and Reconstruction Attack

We are nearly ready to consider an actual set of statistical releases and how this can enable a reconstruction attack, but it will be useful to first consider some details of our attacker.

The attacker's goal is to reconstruct the `income` column as closely as possible (by some definition of closeness), using a combination of the statistical releases and pre-existing knowledge of the database. We will assume for now that the attacker has no pre-existing knowledge, and thus bases their reconstruction only on the statistical releases. A natural way to do this is to attempt to construct a database that is consistent with the releases using a SAT/SMT solver. We will be using [Z3](https://github.com/Z3Prover/z3), but other SAT/SMT solvers could serve the same purpose.

Z3 takes a boolean formula with associated variables and returns whether or not the formula is satisfiable; that is, whether or not there exists a set of variable values that makes the boolean formula `TRUE`. For example, let $A,B$ be boolean variables. The formula $A \land B$ is satisfiable because it is true if $A = B = TRUE$. The formula $A \land B \land \neg B$ is not satisfiable.

This idea can be applied to finding a database that satisfies a set of statistical releases. Say we know that a subgroup has 3 elements, $\{i_1, i_2, i_3\}$ with median income $50,000$, mean income $60,000$, and min income $0$. Note that, because elements within a subgroup are equivalent with respect to the subset of variables that dedine the subset, we are free to index however we wish within the subset. We choose to enforce the element values to be montonically non-decreasing in the index. So, we can represent each statistical release as a statement we'll call an `application`: 

$$ (i_1 \leq i_2 \leq i_3) \land (i_1 = 0) \land (i_2 = 50,000) \land \left( (i_1 + i_2 + i_3)/3 = 60,000 \right). $$

In this case, we get satisfying values of $i_1, i_2$ for free and can find $i_3 = 130,000$ by plugging in $i_1, i_2$ to the last equation. So $\{0, 50,000, 130,000\}$ is a database that is consistent with our statistical release.   

Note that, as long as the set of statistical releases is internally consistent (results do not contradict one another), Z3 should be able to find at least one satisfying assignment.


In [2]:
import z3

# single satisfying assignment
i_1, i_2, i_3 = z3.Ints('i_1 i_2 i_3')
z3.solve(i_1 <= i_2, i_2 <= i_3, i_1 == 0, i_2 == 50_000, z3.Sum(i_1, i_2, i_3) == 180_000)

[i_3 = 130000, i_2 = 50000, i_1 = 0]


In the case above there is only one satisfying assignment, and so the result that Z3 provided is the only database consistent with our statistical releases. Let's now imagine the same case, but without a released median.

In [3]:
# many satisfying assignments
i_1, i_2, i_3 = z3.Ints('i_1 i_2 i_3')

z3.solve(i_1 <= i_2, i_2 <= i_3, i_1 == 0, z3.Sum(i_1, i_2, i_3) == 180_000)
z3.solve(i_1 <= i_2, i_2 <= i_3, i_1 == 0, z3.Sum(i_1, i_2, i_3) == 180_000)
z3.solve(i_1 <= i_2, i_2 <= i_3, i_1 == 0, z3.Sum(i_1, i_2, i_3) == 180_000)


[i_3 = 135119, i_2 = 44881, i_1 = 0]
[i_3 = 147454, i_2 = 32546, i_1 = 0]
[i_3 = 126911, i_2 = 53089, i_1 = 0]


Again we get a satisfying assignment, but this time the assignment is not unique. That is, there are multiple databases consistent with our statistical releases (only one of which is the true underlying data). As a result, the attacker has not learned the entire underlying database with certainty. The more satisfying assignments exist, and the more variability between these assignments, the less the attacker is expected to learn from reconstructing the data.

Now that we have a taste for Z3, we can move to the final stage of setting up the attack, encoding the statistical releases as applications. We treat each individual's income as its own variable and construct applications by asserting that the values of these variables must be consistent with statistical releases, much like we did above (but on a larger scale). For this example, we will assume that the mean, median, min, and max of every subgroup is released, regardless of subgroup size.  

In [4]:
# set applications
applications, priv_applications = rec.get_applications(five_way_interactions, five_way_interactions_names,
                                                plausible_variable_combinations, plausible_variable_combinations_names,
                                                count_dict, priv_count_dict, 
                                                mean_income_dict, priv_mean_income_dict,
                                                median_income_dict, priv_median_income_dict,
                                                min_income_dict, priv_min_income_dict,
                                                max_income_dict, priv_max_income_dict,
                                                elem_dict, priv_elem_dict, lowest_allowable_count = 1)
# remove duplicate applications
applications = list(set(applications))
priv_applications = list(set(priv_applications))

### Attack #1

In [5]:
# initialize solvers
solver, solver_list = rec.applications_to_solver(applications)
priv_solver, priv_solver_list = rec.applications_to_solver(priv_applications) 

# get results (models)
model = rec.check_solution(solver) 
if model:
    print('non-private: sat')
else:
    print('non-private: unsat')

priv_model = rec.check_solution(priv_solver)
if priv_model:
    print('private: sat')
else:
    print('private: unsat')

# attempt to resconstruct data
recon_data = rec.reconstruct_data(model, elem_dict)

non-private: sat
private: unsat


As expected, Z3 produces a satisfying assignment based on the non-private statistical releases. It does not, however, succeed in producing a satisfying assignment for the differentially private statistical releases.

NOTE: NEED TO WRITE THE CODE TO EXAMINE THE DP VERSION FURTHER 

We mentioned earlier that there may exist multiple satisfying assignments and that we can consider the number and variability of these assignments to examine the extent to which an attacker can learn from a satisfying assignment. We will not do that here, instead opting to just compare the assignment to the true underlying data (a luxury the attacker will not have).



In [10]:
# compare original and reconstructed data
orig_data, recon_data, exact, within_2k, within_5k = rec.compare_data(orig_data, recon_data)

print('Of 1000 total incomes:')
print('    {0} incomes reconstructed exactly'.format(exact))
print('    {0} incomes resconstructed within $2,000'.format(within_2k))
print('    {0} incomes resconstructed within $5,000'.format(within_5k))

Of 1000 total incomes:
    112 incomes reconstructed exactly
    383 incomes resconstructed within $2,000
    530 incomes resconstructed within $5,000
